## Create The Trigger Function

In [ ]:
CREATE OR REPLACE FUNCTION update_inventory_function()
RETURNS TRIGGER AS $$
BEGIN
    -- Check if there is enough stock
    IF (SELECT quantity FROM inventory WHERE product_id = NEW.product_id) < NEW.quantity THEN
        RAISE EXCEPTION 'Insufficient stock for product_id %', NEW.product_id;
    ELSE
        -- Decrease the inventory count
        UPDATE inventory
        SET quantity = quantity - NEW.quantity
        WHERE product_id = NEW.product_id;
    END IF;
    RETURN NEW;
END;
$$ LANGUAGE plpgsql;


## Create The Trigger

In [ ]:
CREATE TRIGGER update_inventory
AFTER INSERT ON Order_Items
FOR EACH ROW
EXECUTE FUNCTION update_inventory_function();


## Stored Procedure

In [ ]:
CREATE OR REPLACE PROCEDURE update_customer_status(customer_id INT)
LANGUAGE plpgsql
AS $$
BEGIN
    -- Calculate the total order value for the customer
    DECLARE total_order_value NUMERIC;
    SELECT SUM(amount) INTO total_order_value
    FROM Orders
    WHERE customer_id = customer_id;

    -- Update the customer status based on the total order value
    IF total_order_value > 10000 THEN
        UPDATE Customers
        SET status = 'VIP'
        WHERE id = customer_id;
    ELSE
        UPDATE Customers
        SET status = 'Regular'
        WHERE id = customer_id;
    END IF;
END;
$$;
